# Proyecto Estructura y Bases de Datos

*   Antonia Morales
*   Ángel Paisano
*   Martín Raffo
*   Esteban Sánchez





# Primera descripción

En el presente proyecto se utiliza la API astroquery para cargar los datos de Gaia usando ADQL (Astronomical Data Query Language) que es el lenguaje que usa Gaia; este es idéntico a SQL, que fue visto en clases, pero con funciones integradas para trabajar datos astronómicos. En principio los datos se almacenan en una tabla astropy, y luego son transformados a dataframes de Pandas para hacerlos accesibles. Finalmente, a partir de los dataframes, se crean tablas en SQL y se realizan diversas consultas con la información contenida en ellas, junto con gráficos que contribuyen a la interpretación de los datos en su contexto.

Para el proyecto, es conveniente trabajar con SQL debido a la naturaleza de los datos: éstos son homogéneos, fácilmente organizables en tablas de tipo relacional y no variables en el tiempo. Cabe destacar que, si bien TaQL también es utilizado para bases de datos relacionales, tampoco es una elección adecuada en este contexto, puesto que su enfoque es trabajar información proveniente de observaciones en radio, mientras que los datos de Gaia se encuentran en la banda del óptico.


# Gaia

In [ ]:
#Instalando astroquery
!pip install astroquery

In [ ]:
#Se importan librerías
from astroquery.gaia import Gaia
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Se aumenta el timeout para que pueda correr tranquilamente el código
Gaia.TIMEOUT = 1e6

In [ ]:
#Se cargan las tablas
tables = Gaia.load_tables(only_names=True)

In [ ]:
#Imprimir nombres de tablas
for table in tables:
    print(table.name)

## ADQL
La muestra de la gran nube de magallanes (siglas en inglés: LMC) se obtiene usando una selección con un radio de 20° con $α,δ$ (81.28°,-67.78°). El trabajo se enfocará en esta región del cielo para procesar datos de la galaxia anteriormente mencionada, estudiando las posiciones en el cielo de las fuentes astronómicas (sources), sus movimientos propios, y las magnitudes en distintos filtros. Este paso está basado en el artículo científico de Luri et al (2021) que se dedica a seleccionar datos de Gaia EDR3 en la región de la LMC.

Después de hacer las consultas pertinentes en Gaia DR2 y DR3 para obtener los datos de las fuentes en las Nubes de Magallanes, se guardan los resultados en tablas de Astropy.

In [ ]:
# Consulta a Gaia y conversión
job0 = Gaia.launch_job_async("""SELECT TOP 100000 source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag,
                      phot_rp_mean_mag FROM gaiadr2.gaia_source
                      WHERE DISTANCE(POINT(81.28, -69.78), POINT(ra, dec)) < 20
                      AND gaiadr2.gaia_source.parallax IS NOT NULL""")
tabla_gaiadr2 = job0.get_results()  # Resultado en formato Table de Astropy

In [ ]:
tabla_gaiadr2

In [ ]:
#Obtención de RUWE para Gaia DR2
job1 = Gaia.launch_job_async("""SELECT TOP 100000 gaiadr2.gaia_source.source_id, gaiadr2.ruwe.ruwe
                            FROM gaiadr2.ruwe JOIN gaiadr2.gaia_source
                            ON gaiadr2.ruwe.source_id = gaiadr2.gaia_source.source_id
                            WHERE DISTANCE(POINT(81.28, -69.78),POINT(ra, dec)) < 20
                            AND gaiadr2.gaia_source.parallax IS NOT NULL""")

tabla_ruwe_dr2 = job1.get_results()

tabla_ruwe_dr2

In [ ]:
# Consulta a Gaia y conversión DR3
job2 = Gaia.launch_job_async("""SELECT TOP 100000 source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag,
                      phot_rp_mean_mag, ruwe FROM gaiadr3.gaia_source
                      WHERE DISTANCE(POINT(81.28, -69.78),POINT(ra, dec)) < 20
                      AND gaiadr3.gaia_source.parallax IS NOT NULL""")
tabla_gaiadr3 = job2.get_results()  # Resultado en formato Table de Astropy

In [ ]:
tabla_gaiadr3

## Pandas

Ya teniendo las tablas de Astropy, se transforman a Dataframes de Pandas para eventualmente poblar con éstas una base de datos SQL.


In [ ]:
df_gaiadr2 = tabla_gaiadr2.to_pandas()
df_gaiadr2 = df_gaiadr2.rename(columns={"SOURCE_ID": "source_id"})
df_gaiadr2["ruwe"] = tabla_ruwe_dr2["ruwe"]

In [ ]:
df_gaiadr2

In [ ]:
df_gaiadr3 = tabla_gaiadr3.to_pandas()
df_gaiadr3 = df_gaiadr3.rename(columns={"SOURCE_ID": "source_id"})

In [ ]:
df_gaiadr3

# SQL

Acto seguido, se define una función que permitirá la elaboración de tablas tipo SQL a partir de los Dataframes resultantes del paso anterior. Dicha función es ejecutada para DR2 y DR3, a fin de poder realizar consultas y gráficos.

In [ ]:
# Función para crear las tablas de gaia
def crear_tabla(tabla, nombre_tabla, db_name):
  """
  Esta función construye...

  """
  db = sqlite3.connect(db_name)
  cur = db.cursor()

  ############################

  cur.execute(f"""
        CREATE TABLE {nombre_tabla} (
            source_id INTEGER PRIMARY KEY AUTOINCREMENT,
            ra REAL,
            dec REAL,
            parallax REAL,
            pmra REAL,
            pmdec REAL,
            phot_g_mean_mag REAL,
            phot_bp_mean_mag REAL,
            phot_rp_mean_mag REAL,
            ruwe REAL
        );
    """)

  ####################################################
  for i, row in tabla.iterrows():
        cur.execute(f"""
            INSERT OR IGNORE INTO {nombre_tabla} (source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, ruwe)
            VALUES (?, ?, ?, ?, ?, ?, ?, ? ,?, ?);
        """, (row['source_id'], row['ra'], row['dec'], row['parallax'], row['pmra'], row['pmdec'], row['phot_g_mean_mag'], row['phot_bp_mean_mag'], row['phot_rp_mean_mag'], row['ruwe']))

  db.commit()

  # Verificación:
  res = cur.execute(f"PRAGMA table_info({nombre_tabla})")
  if res.fetchall() != []:
    print("Tabla creada correctamente")
    print(res.fetchall())
  else:
    print("Error al crear la tabla")

  db.close()

In [ ]:
#Creación de tablas con la función

crear_tabla(df_gaiadr2, 'gaiadr2', 'gaiadr2.db')

In [ ]:
crear_tabla(df_gaiadr3, 'gaiadr3', 'gaiadr3.db')

# Querys SQL



In [ ]:
db = sqlite3.connect('gaiadr2.db')
cur = db.cursor()
cur.execute("ATTACH DATABASE 'gaiadr3.db' AS gaiadr3;")

In [ ]:
res = cur.execute(f"""PRAGMA table_info(gaiadr2)""")
res.fetchall()

In [ ]:
query0 = cur.execute(f"""SELECT COUNT(*) FROM gaiadr2""")
query0.fetchall()

In [ ]:
query1= cur.execute(f"""SELECT COUNT(*) FROM
                      (SELECT gaiadr2.source_id AS source_id_gaiadr2,
                      gaiadr3.source_id AS source_id_gaiadr3 FROM gaiadr2
                      JOIN gaiadr3 ON gaiadr2.source_id=gaiadr3.source_id)""")
query1.fetchall()

In [ ]:
query2= cur.execute(f"""SELECT ROW_NUMBER() OVER(ORDER BY gaiadr2.source_id) AS row_number,
                          gaiadr2.source_id AS source_id_gaiadr2,
                          gaiadr3.source_id AS source_id_gaiadr3
                        FROM gaiadr2
                        JOIN gaiadr3
                        ON gaiadr2.source_id = gaiadr3.source_id;""")

rows = query2.fetchall()
for row in rows:
    print(f"Numero de Columna: {row[0]}, GAIADR2 Source ID: {row[1]}, GAIADR3 Source ID: {row[2]}")

In [ ]:
query3 = cur.execute("""SELECT COUNT(*) AS n FROM (SELECT gaiadr2.source_id, gaiadr2.phot_rp_mean_mag FROM gaiadr2
                        JOIN gaiadr3 ON gaiadr2.source_id = gaiadr3.source_id
                        WHERE gaiadr2.phot_rp_mean_mag < 20)""")
query3.fetchall()

In [ ]:
query4 = cur.execute("""CREATE TABLE filtro_dr22 AS SELECT * FROM gaiadr2 WHERE gaiadr2.ruwe < 1.4 ORDER BY ruwe DESC""")
query4.fetchall()

# Gráficos

In [ ]:
plt.style.use('dark_background')

In [ ]:
# Histograma de ...
nan_count_g = df_gaiadr2['phot_g_mean_mag'].isna().sum()
print(nan_count_g)
nan_count_bp = df_gaiadr2['phot_bp_mean_mag'].isna().sum()
print(nan_count_bp)

plt.hist(df_gaiadr2['phot_bp_mean_mag'],bins=100)
plt.xlim(16,24)
plt.axvline(20,color='red',linestyle='--')
plt.xlabel('Magnitud BP')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
#Variables (DR2)
ruwe_dr2 = df_gaiadr2['ruwe']
pmra_dr2 = df_gaiadr2['pmra']
pmdec_dr2 = df_gaiadr2['pmdec']

In [ ]:
#Variables que utilizaremos para graficar
ra = df_gaiadr3['ra']
dec = df_gaiadr3['dec']
pmra = df_gaiadr3['pmra']
pmdec = df_gaiadr3['pmdec']
mag_bp = df_gaiadr3['phot_bp_mean_mag']
mag_rp = df_gaiadr3['phot_rp_mean_mag']
mag_g = df_gaiadr3['phot_g_mean_mag']
ruwe_dr3 = df_gaiadr3['ruwe']

In [ ]:
fig = plt.figure(figsize=(14,6))
#fig.subplots_adjust(left=0.1, bottom=0.1, top=0.97, right =0.97)

ax1 = fig.add_subplot(121)

ax1.scatter(ra, dec, c='cyan', marker='*',s=1, edgecolors= "none") #c=brillo, cmap='plasma',s=(1/brillo)*40)
ax1.scatter(81.28, -69.78, marker="+", facecolor="red",label='Centro aparente', s=40)
ax1.set_xlabel("RA")
ax1.set_ylabel("DEC")
ax1.set_title(" ra vs dec de la Gran Nube de Magallanes")
ax1.grid(ls='--', alpha=0.3)
ax1.legend()

ax2= fig.add_subplot(122)

ax2.scatter(pmra, pmdec, c='cyan', marker='*',s=1, edgecolors= "none") #c=brillo, cmap='plasma',s=(1/brillo)*40)
ax2.set_xlabel("pmra")
ax2.set_ylabel("pmdec")
ax2.set_title("pmra vs pmdec de la Gran Nube de Magallanes")
ax2.grid(ls='--', alpha=0.3)

plt.show()

In [ ]:
#Histograma de RUWE (DR2)
plt.hist(ruwe_dr2, bins = 500)
plt.axvline(1.4, color = 'red')
plt.axhline(10000, color = 'blue')
plt.xlim(0,6)
plt.show()

In [ ]:
#Histograma de RUWE (DR3)
plt.hist(ruwe_dr3, bins = 500)
plt.axvline(1.4, color = 'red')
plt.axhline(5000, color = 'blue')
plt.xlim(0,6)
plt.show()

In [ ]:
#Histograma 2D

plt.hist2d(ra, dec, bins = 100)
plt.title("Histograma 2D de RA vs DEC de la Gran Nube de Magallanes")
plt.xlabel("RA")
plt.ylabel("DEC")
plt.show()

In [ ]:
#Histograma 2D (movimientos propios)

plt.hist2d(pmra, pmdec, bins = 100)
plt.show()

## Diagrama color magnitud test
BP-RP vs G

In [ ]:
#añadir errorbars y colorbars

fig = plt.figure(1,figsize=(6,6))
ax1= fig.add_subplot(111)
ax1.scatter(mag_bp - mag_rp, mag_g,c='darkred',marker='*',s=10, edgecolors= "none") #c=brillo, cmap='plasma',s=(1/brillo)*40)
#ax1.scatter((mag_bp-mag_rp)[i], mag_g[i],marker="*",edgecolor="none", facecolor="red",s=10)
ax1.set_xlabel("BP-RP")
ax1.set_ylabel("G")
ax1.set_title("Diagrama color magnitud Gran Nubre de Magallanes")
ax1.grid(ls='--', alpha=0.3)
#plt.ylim(16,21.5)
plt.gca().invert_yaxis()

plt.show()